In [1]:
import pandas as pd
import sqlite3
from sqlalchemy import create_engine

In [2]:
engine = create_engine('sqlite:///ts.sqlite3', echo=True)
sqlite_connection = engine.connect()

2021-06-27 22:51:27,323 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2021-06-27 22:51:27,323 INFO sqlalchemy.engine.base.Engine ()
2021-06-27 22:51:27,332 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2021-06-27 22:51:27,332 INFO sqlalchemy.engine.base.Engine ()


In [3]:
from sqlalchemy import inspect
inspector = inspect(engine)

# Get table information
print(inspector.get_table_names())

2021-06-27 22:51:27,348 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2021-06-27 22:51:27,356 INFO sqlalchemy.engine.base.Engine ()
['ts1', 'ts_pred']


In [4]:
# Get column information
print(inspector.get_columns('ts1'))

2021-06-27 22:51:27,372 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_xinfo("ts1")
2021-06-27 22:51:27,375 INFO sqlalchemy.engine.base.Engine ()
[{'name': 'index', 'type': BIGINT(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}, {'name': 'time', 'type': DATETIME(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}]


In [5]:
df = pd.read_sql_query("SELECT * from ts1", engine)
df

2021-06-27 22:51:27,393 INFO sqlalchemy.engine.base.OptionEngine SELECT * from ts1
2021-06-27 22:51:27,393 INFO sqlalchemy.engine.base.OptionEngine ()


,index,time
0,0,2017-10-15 00:00:00.000000
1,1,2017-10-07 00:00:00.000000
2,2,2017-08-20 00:00:00.000000
3,3,2017-03-31 00:00:00.000000
4,4,2017-03-05 00:00:00.000000
...,...,...
7440,7507,2018-08-04 00:00:00.000000
7441,7508,2020-01-02 00:00:00.000000
7442,7509,2019-02-01 00:00:00.000000
7443,7510,2018-08-11 00:00:00.000000


In [6]:
sqlite_connection.close()

In [7]:
df['time'] = pd.to_datetime(df['time'])

In [8]:
df.drop(columns='index',inplace = True)
df


,time
0,2017-10-15
1,2017-10-07
2,2017-08-20
3,2017-03-31
4,2017-03-05
...,...
7440,2018-08-04
7441,2020-01-02
7442,2019-02-01
7443,2018-08-11


In [9]:

from warnings import simplefilter

import numpy as np
import pandas as pd

from sktime.datasets import load_airline
from sktime.forecasting.arima import ARIMA, AutoARIMA
from sktime.forecasting.base import ForecastingHorizon
from sktime.forecasting.compose import (
    EnsembleForecaster,
    MultiplexForecaster,
    TransformedTargetForecaster,
    make_reduction,
)
from sktime.forecasting.exp_smoothing import ExponentialSmoothing
from sktime.forecasting.model_evaluation import evaluate
from sktime.forecasting.model_selection import (
    ExpandingWindowSplitter,
    ForecastingGridSearchCV,
    SlidingWindowSplitter,
    temporal_train_test_split,
)
from sktime.forecasting.naive import NaiveForecaster
from sktime.forecasting.theta import ThetaForecaster
from sktime.forecasting.trend import PolynomialTrendForecaster
from sktime.performance_metrics.forecasting import (
    MeanAbsolutePercentageError,
    mean_absolute_percentage_error,
)
from sktime.transformations.series.detrend import Deseasonalizer, Detrender
from sktime.utils.plotting import plot_series

simplefilter("ignore", FutureWarning)
%matplotlib inline

In [10]:
df = df[df.time.dt.year>2017]

In [11]:
ts1 = df.resample('W-MON',on='time').count()

In [12]:
ts1 = ts1[:-1]
ts1 = ts1[1:]


In [13]:
ts1.index = ts1.index.to_period()
y = ts1.time
max_date = max(ts1.index).strftime('%Y-%m-%d')


In [14]:
fh = ForecastingHorizon(
    pd.PeriodIndex(pd.date_range('2020-09-03', periods=50, freq="W-MON")), is_relative=False
 )

In [15]:
forecaster = ExponentialSmoothing(trend="add", seasonal="additive", sp=52)
forecaster.fit(y)
y_pred = forecaster.predict(fh)
#mean_absolute_percentage_error(y_pred, y)

In [16]:
y_pred.index = y_pred.index.to_timestamp()

In [17]:
pred = pd.DataFrame(y_pred).reset_index()
pred['method']='ExponentialSmoothing'
pred.columns=[['date','value','method']]

In [19]:
y.index = y.index.to_timestamp()

In [20]:
fact = pd.DataFrame(y)
fact.rename(columns={'time':'val'},inplace=True)

In [21]:
fact.reset_index(inplace=True)

In [22]:
fact['method']='fact'

In [23]:
fact.columns=[['date','value','method']]

In [24]:
full = pd.concat([fact,pred])

In [25]:
full

,date,value,method
0,2018-01-02,21.000000,fact
1,2018-01-09,28.000000,fact
2,2018-01-16,32.000000,fact
3,2018-01-23,25.000000,fact
4,2018-01-30,33.000000,fact
...,...,...,...
45,2021-07-13,51.144138,ExponentialSmoothing
46,2021-07-20,50.477456,ExponentialSmoothing
47,2021-07-27,54.144093,ExponentialSmoothing
48,2021-08-03,48.144096,ExponentialSmoothing


In [26]:

sqlite_connection = engine.connect()
sqlite_table = "ts_pred"
full.to_sql(sqlite_table, sqlite_connection, if_exists='replace')

2021-06-27 22:52:08,182 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("ts_pred")
2021-06-27 22:52:08,184 INFO sqlalchemy.engine.base.Engine ()
2021-06-27 22:52:08,186 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("ts_pred")
2021-06-27 22:52:08,187 INFO sqlalchemy.engine.base.Engine ()
2021-06-27 22:52:08,190 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2021-06-27 22:52:08,192 INFO sqlalchemy.engine.base.Engine ()
2021-06-27 22:52:08,194 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_xinfo("ts_pred")
2021-06-27 22:52:08,195 INFO sqlalchemy.engine.base.Engine ()
2021-06-27 22:52:08,197 INFO sqlalchemy.engine.base.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type = 'table'
2021-06-27 22:52:08,199 INFO sqlalchemy.engine.base.Engine ('ts_pred',)
2021-06-27 22:52:08,201 INFO sqlalchemy.engine.base.Engine PRAGMA main.foreign_key_list("ts_

In [27]:
sqlite_connection.close()